In [1]:
# from dask.distributed import Client, progress
import dask.dataframe as dd
import dask
import pandas as pd
import csv
from glob import glob

In [8]:
# loading in through pandas
df = pd.read_csv("/mnt/research/CMSE495-SS24-ICER/slurm_usage/DID_FINAL_SLURM_OCT_2023.csv", delimiter = "|", nrows = 1e5)
df = df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
df = df[df["State"] == "COMPLETED"]
df.head(5)

,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,AssocID,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
4,59062820,user_188,group_046,2023-05-08T09:58:20,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
5,59062828,user_188,group_046,2023-05-08T09:58:20,2024-01-01T01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
6,59062836,user_188,group_046,2023-05-08T09:58:20,2024-01-01T01:00:49,2024-01-01T01:00:55,00:00:06,COMPLETED,account_017,assocID_676,...,240,40,40,8G,NaN,1,1,skl-030,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
7,59062844,user_188,group_046,2023-05-08T09:58:21,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"
8,59062852,user_188,group_046,2023-05-08T09:58:21,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,assocID_676,...,360,40,40,8G,NaN,1,1,amr-207,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [8]:
# from dask.distributed import Client, progress
import dask.dataframe as dd
import gc
import pandas as pd
import time

# loading in data
print("Loading in data")
slurm = dd.read_csv("/mnt/research/CMSE495-SS24-ICER/slurm_usage/DID_FINAL_SLURM_OCT_2023.csv",
                    delimiter="|", dtype={'MaxRSS': 'object'}).reset_index()
slurm = slurm.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

# repartition data
slurm = slurm.repartition(npartitions=4)
print(f"Number of records: {len(slurm)}")

Loading in data
Number of records: 6376308


In [9]:
i=1
for part in slurm.partitions:
    i+=1
print(i)

5


In [10]:
slurm.head()

,index,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
0,0,31496544,user_679,group_121,2023-03-21T11:13:45,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,28,0,21000M,<NA>,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",<NA>
1,1,31497932,user_679,group_121,2023-03-21T11:31:18,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,28,0,21000M,<NA>,1,1,None assigned,"billing=3192,cpu=28,gres/gpu=4,mem=21000M,node=1",<NA>
2,2,31993628,user_105,group_114,2023-03-22T18:19:12,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,12,0,150G,<NA>,1,1,None assigned,"billing=23347,cpu=12,gres/gpu=8,mem=150G,node=1",<NA>
3,3,39087660,user_652,group_054,2023-04-04T13:09:10,Unknown,Unknown,00:00:00,PENDING,account_017,...,0,640,0,20G,<NA>,10,10,None assigned,"billing=3112,cpu=640,mem=20G,node=10",<NA>
4,4,59062820,user_188,group_046,2023-05-08T09:58:20,2024-01-01T00:58:57,2024-01-01T00:59:06,00:00:09,COMPLETED,account_017,...,360,40,40,8G,<NA>,1,1,skl-029,"billing=1245,cpu=40,mem=8G,node=1","billing=1245,cpu=40,mem=8G,node=1"


In [24]:
for npart,sl_part in enumerate(slurm.partitions):

    # finding batch data
    batch_extern = sl_part.loc[sl_part.User=="user_258"]
    batch = batch_extern[batch_extern.MaxRSS.notnull()]
    batch = batch.loc[batch.Elapsed != '00:00:00']
    bsub = batch.loc[:,["JobID","CPUTimeRAW","MaxRSS"]]

    # creating list of jobs
    joblist = bsub["JobID"].drop_duplicates()

    # cleaning up 0 second jobs and batch/extern jobs
    slurm_clean = sl_part.loc[(sl_part.Elapsed != '00:00:00') & (sl_part.User!="user_258")].drop_duplicates().reset_index()

    start = time.time()
    for i,j in enumerate(joblist):

        if i%20==0:
            gc.collect()

        print("Current Job:",str(j),"at",str(time.time()-start), flush=True)
        job = slurm_clean.where(cond=(slurm_clean["JobID"] == j))

        if len(job["JobID"]) == 1:
            slurm_clean["MaxRSS"] = slurm["MaxRSS"].where(slurm_clean["JobID"] == j, 
                                                          bsub.loc[bsub["JobID"]==j,"MaxRSS"])
        else:
            for cputime in job["CPUTimeRAW"]:

                # if there are multiple instances where the CPU time is the same, the memory usage is the same
                mem_used = bsub.loc[bsub["CPUTimeRAW"]==cputime,"MaxRSS"].reset_index()
                ndf = slurm.where(cond=(slurm_clean["CPUTimeRAW"] == cputime), 
                                  other=mem_used.loc[0,"MaxRSS"])
                print("here?")
    
    print(npart,"Complete")


Current Job: 87461884 at 7.998202323913574


ValueError: Not all divisions are known, can't align partitions. Please use `set_index` to set the index.. If you don't want the partitions to be aligned, and are calling `map_partitions` directly, pass `align_dataframes=False`.

In [25]:
# slurm_clean.partitions
df = slurm_clean.where(slurm_clean["CPUTimeRAW"] == cputime, "beans").compute()

In [23]:
df["CPUTimeRAW"].dropna()

0         360.0
3         360.0
4         360.0
5         360.0
13        360.0
          ...  
485136    360.0
485144    360.0
485145    360.0
486514    360.0
486515    360.0
Name: CPUTimeRAW, Length: 228, dtype: float64

In [4]:
filenames = glob('/mnt/scratch/tairaeli/cmse_dat/slurm0_sec_jobs/part**.csv')
with open('/mnt/scratch/tairaeli/cmse_dat/clean_slurm_0sec.csv', 'w') as out:
    for fn in filenames:
        with open(fn) as f:
            out.write(f.read())

In [2]:
cleanish_slurm = pd.read_csv("/mnt/scratch/tairaeli/cmse_dat/clean_slurm_4.csv",nrows = 100)
cleanish_slurm.head()

,Unnamed: 0,JobID,User,Group,Submit,Start,End,Elapsed,State,Account,...,CPUTimeRAW,ReqCPUS,AllocCPUS,ReqMem,MaxRSS,ReqNodes,NNodes,NodeList,ReqTRES,AllocTRES
0,458,93780800,user_690,group_082,2023-10-09T10:26:30,2023-10-09T10:39:37,2023-10-09T10:47:45,00:08:08,COMPLETED,account_066,...,1952,4,4,20G,NaN,1,1,lac-240,"billing=3112,cpu=4,mem=20G,node=1","billing=3112,cpu=4,mem=20G,node=1"
1,461,93780804,user_624,group_154,2023-10-09T10:26:30,2023-10-09T10:30:05,2023-10-09T13:08:09,02:38:04,COMPLETED,account_025,...,9484,1,1,4G,NaN,1,1,amr-148,"billing=622,cpu=1,mem=4G,node=1","billing=622,cpu=1,mem=4G,node=1"
2,2464,93780808,user_690,group_082,2023-10-09T10:26:30,2023-10-09T10:39:37,2023-10-09T10:47:11,00:07:34,COMPLETED,account_066,...,1816,4,4,20G,NaN,1,1,lac-294,"billing=3112,cpu=4,mem=20G,node=1","billing=3112,cpu=4,mem=20G,node=1"
3,2467,93780812,user_624,group_154,2023-10-09T10:26:30,2023-10-09T10:30:05,2023-10-09T14:30:09,04:00:04,TIMEOUT,account_025,...,14404,1,1,4G,NaN,1,1,amr-148,"billing=622,cpu=1,mem=4G,node=1","billing=622,cpu=1,mem=4G,node=1"
4,3316,93780816,user_690,group_082,2023-10-09T10:26:30,2023-10-09T10:39:47,2023-10-09T11:01:39,00:21:52,COMPLETED,account_066,...,5248,4,4,20G,NaN,1,1,lac-240,"billing=3112,cpu=4,mem=20G,node=1","billing=3112,cpu=4,mem=20G,node=1"
